In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Phase 2: regime features
# -------------------------------
from phase_2.scripts.strategies.meta.meta_allocator_v1 import (
    build_regime_features,
)

# -------------------------------
# Phase 3: strategy gates
# -------------------------------
from phase_3.strategies.meta.strategy_gates_v1 import (
    build_strategy_gates_v1,
    DEFAULT_GATE_PARAMS,
)


In [ ]:
spy = pd.read_parquet("../../phase_2/data/processed/spy_daily.parquet")

spy["date"] = pd.to_datetime(spy["date"])
spy = spy.sort_values("date").reset_index(drop=True)
spy["year"] = spy["date"].dt.year

spy.head()


In [ ]:
regime = build_regime_features(spy)

regime[["date", "mom_60", "vol_20", "drawdown_60"]].head(10)

In [ ]:
gates = build_strategy_gates_v1(
    regime_df=regime,
    p=DEFAULT_GATE_PARAMS,
)

gates.head(10)

In [ ]:
df = (
    regime[["date", "mom_60", "vol_20"]]
    .merge(gates, on="date", how="left")
)

df.head(10)


In [ ]:
summary = {
    "trend_allowed_pct": df["trend_allowed"].mean(),
    "meanrev_allowed_pct": df["meanrev_allowed"].mean(),
    "both_disabled_pct": ((~df["trend_allowed"]) & (~df["meanrev_allowed"])).mean(),
}

pd.Series(summary)


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# mom_60
ax[0].plot(df["date"], df["mom_60"], label="mom_60")
ax[0].axhline(0.0, color="black", linestyle="--", linewidth=1)
ax[0].set_title("SPY 60d Momentum")
ax[0].grid(True)

# trend gate
ax[1].plot(
    df["date"],
    df["trend_allowed"].astype(int),
    drawstyle="steps-post",
    label="trend_allowed",
)
ax[1].set_yticks([0, 1])
ax[1].set_yticklabels(["OFF", "ON"])
ax[1].set_title("Trend Strategy Gate")
ax[1].grid(True)

plt.show()


In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

# mom_60
ax[0].plot(df["date"], df["mom_60"], label="mom_60")
ax[0].axhline(0.0, color="black", linestyle="--", linewidth=1)
ax[0].set_title("SPY 60d Momentum")
ax[0].grid(True)

# vol_20
ax[1].plot(df["date"], df["vol_20"], label="vol_20")
ax[1].axhline(0.60, color="red", linestyle="--", linewidth=1, label="panic vol")
ax[1].set_title("SPY 20d Volatility (annualized)")
ax[1].grid(True)

# meanrev gate
ax[2].plot(
    df["date"],
    df["meanrev_allowed"].astype(int),
    drawstyle="steps-post",
    label="meanrev_allowed",
)
ax[2].set_yticks([0, 1])
ax[2].set_yticklabels(["OFF", "ON"])
ax[2].set_title("Mean-Reversion Strategy Gate")
ax[2].grid(True)

plt.show()

In [ ]:
df.loc[df["vol_20"] > 0.60, ["date", "vol_20"]].head(10)
